In [4]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [5]:
#
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_85d12d4f65ee47788c560ea38d005220 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_85d12d4f65ee47788c560ea38d005220 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_85d12d4f65ee47788c560ea38d005220 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='u761p0twCVQN2UjDC-y8vqA7YrR8RemGUR0Z73DaeC7a',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_85d12d4f65ee47788c560ea38d005220)

body = client_85d12d4f65ee47788c560ea38d005220.get_object(Bucket='predictingtheenergyoutputofwindtu-donotdelete-pr-xjrhxpjy5fzsti',Key='T1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset= pd.read_csv(body)
dataset.head()
#df = pd.read_csv("C:/Users/DELL/Desktop/MINI PROJECT/CSE-008/MINI PROJECT/T1.csv")

ModuleNotFoundError: No module named 'botocore'

In [ ]:
#

In [ ]:
dataset.rename(columns={'Date/Time':'Time',
                   'LV ActivePower (kW)':'ActivePower(kW)',
                   "Wind Speed (m/s)":"WindSpeed(m/s)",
                   "Wind Direction (°)":"Wind_Direction"},
                    inplace=True)

In [ ]:
sns.pairplot(dataset)

In [ ]:
corr = dataset.corr()
plt.figure(figsize=(10, 8))

In [ ]:
ax = sns.heatmap(corr, vmin = -1, vmax = 1, annot = True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()
corr

In [ ]:
'''
The heat map clearly tells us that there's no realtion between wind direction and 
the Power generated but Wind speed, Theoritical power and Actual power generated 
have a very positive correlation
'''

In [ ]:
#df.drop(['Wind_Direction'],axis=1,inplace = True)
dataset["Time"] = pd.to_datetime(dataset["Time"], format = "%d %m %Y %H:%M", errors = "coerce")
dataset

In [ ]:
y = dataset['ActivePower(kW)'] #'Theoretical_Power_Curve (KWh)'
X = dataset[['Theoretical_Power_Curve (KWh)','WindSpeed(m/s)']]#'ActivePower(kW)'

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score

In [ ]:
forest_model = RandomForestRegressor(max_leaf_nodes =500, random_state=1)
forest_model.fit(train_X, train_y)

In [ ]:
power_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, power_preds))
print(r2_score(val_y,power_preds))
joblib.dump(forest_model, "power_prediction.sav")

In [ ]:
!pip install ibm_watson_machine_learning
!pip install APIClient

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                     "url":"https://us-south.ml.cloud.ibm.com",
                     "apikey":"-F0oHNJ9jo9hYnF5fznLzlYSp8TLCaUgnESEjYJPvtTL"
}
client=APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name )['metadata']['id'])

In [ ]:
client=APIClient(wml_credentials)

In [ ]:
space_uid = guid_from_space_name(client, 'models')
print('space UID =' + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model=forest_model,meta_props={
client.repository.ModelMetaNames.NAME:"churn_modeling",
client.repository.ModelMetaNames.TYPE:"scikit-learn_0.23",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid })
model_id = client.repository.get_model_uid(model_details)

In [ ]:
model_id 